In [ ]:
import pygame
import random
import math


# Determine the constant sizes
size_of_board = 25
size_of_square = 15
height = 20
screen_size = (size_of_board * size_of_square, size_of_board * size_of_square + height)


# Creates a Node class
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.distance_from_start = 0
        self.heuristic = 5
        self.total_cost = 0

    # Determines whether or not the two coordinate values are equal
    def equal(self, compare):
        return self.x == compare.x and self.y == compare.y


class Algorithm:
    def __init__(self, grid):
        self.grid = grid
        self.queue = []
        self.explored_set = []
        self.path = []

    def get_initstate_and_fruit(self, snake):
        return Node(snake.head[0], snake.head[1]), Node(snake.apple[0], snake.apple[1])

    # Total distance traveled
    def manhattan_distance(self, nodeA, nodeB):
        distance_1 = abs(nodeA.x - nodeB.x)
        distance_2 = abs(nodeA.y - nodeB.y)
        return distance_1 + distance_2

    def get_path(self, node):
        if node.parent == None:
            return node

        while node.parent.parent != None:
            self.path.append(node)
            node = node.parent
        return node

    def inside_body(self, snake, node):
        for body in snake.body:
            if body[0] == node.x and body[1] == node.y:
                return True
        return False

    def outside_boundary(self, node):
        if 1 < node.x < size_of_board - 2:
            return False
        if 1 < node.y < size_of_board - 2:
            return False
        return True

    def get_neighbors(self, head):
        i = int(head.x)
        j = int(head.y)

        neighbors = []
        # left [i-1, j]
        if i > 1:
            neighbors.append(self.grid[i - 1][j])

        # right [i+1, j]
        if i < size_of_board - 2:
            neighbors.append(self.grid[i + 1][j])

        # top [i, j-1]
        if j > 1:
            neighbors.append(self.grid[i][j - 1])

        # bottom [i, j+1]
        if j < size_of_board - 2:
            neighbors.append(self.grid[i][j + 1])


        return neighbors


class A_STAR(Algorithm):
    def __init__(self, grid):
        super().__init__(grid)

    def run_algorithm(self, snake):
        # clear everything
        self.queue = []
        self.explored_set = []
        # to remove

        self.path = []

        snake_head, fruit = self.get_initstate_and_fruit(snake)

        self.new_path = None

        # open list
        self.queue.append(snake_head)

        # while we have states in open list
        while len(self.queue) > 0:
            # get node with lowest f(n)
            lowest_index = 0
            for i in range(len(self.queue)):
                if self.queue[i].total_cost < self.queue[lowest_index].total_cost:
                    lowest_index = i

            lowest_node = self.queue.pop(lowest_index)

            # check if it's the goal state
            if lowest_node.equal(fruit):
                self.new_path = self.get_path(lowest_node)
                return self.new_path

            self.explored_set.append(lowest_node)  # mark visited

            neighbors = self.get_neighbors(lowest_node)  # get neighbors

            # for each neighbor
            for neighbor in neighbors:

                # check if path is inside snake, outside boundary, or already visited
                if (
                    self.inside_body(snake, neighbor)
                    or self.outside_boundary(neighbor)
                    or neighbor in self.explored_set
                ):
                    continue  # skip this path
                # Updates the distance cost from the start
                temp = lowest_node.distance_from_start + 1

                best = False  # assuming neighbor path is better

                if neighbor not in self.queue:  # first time visiting
                    neighbor.heuristic = self.manhattan_distance(fruit, neighbor)
                    self.queue.append(neighbor)
                    best = True
                elif (
                    lowest_node.distance_from_start < neighbor.distance_from_start
                ):  # has already been visited
                    best = (
                        True  # but had a worse distance from the start, now it's better
                    )

                if best:
                    neighbor.parent = lowest_node
                    neighbor.distance_from_start = temp
                    neighbor.total_cost = (
                        neighbor.distance_from_start + neighbor.heuristic
                    )
                    
        return None


class Snake:
    def __init__(self):
        self.body = [
            [size_of_board - 2, 1],
            [size_of_board - 3, 2],
            [size_of_board - 4, 3],
        ]
        self.head = self.body[0]
        self.apple = [
            random.randint(2, size_of_board - 2),
            random.randint(2, size_of_board - 2),
        ]
        self.direction = (1, 0)
        self.grow = False
        self.wall = []
        self.astar = None
        for i in range(size_of_board):
            self.wall.insert(0, [i, 0])
            self.wall.insert(0, [0, i])
            self.wall.insert(0, [i, size_of_board - 1])
            self.wall.insert(0, [size_of_board - 1, i])

    def update(self, grid):
        astar = A_STAR(grid)
        self.astar = astar
        next_move = astar.run_algorithm(self)
        if next_move:
            new_direction = (next_move.x - self.head[0], next_move.y - self.head[1])

            # Update the direction only if it is not the opposite of the current direction
            if (new_direction[0], new_direction[1]) != (
                -self.direction[0],
                -self.direction[1],
            ):
                self.direction = new_direction

        new_head = [self.head[0] + self.direction[0], self.head[1] + self.direction[1]]
        self.body.insert(0, new_head)
        self.head = new_head

        if not self.grow:
            self.body.pop()
        else:
            # print('body = \t', self.body, '\napple = ', self.apple, '\nself.head = ', self.head)

            self.grow = False

        if self.head == self.apple:
            self.apple = [
                random.randint(2, size_of_board - 2),
                random.randint(2, size_of_board - 2),
            ]
            while self.apple in self.body or self.apple in self.wall:
                self.apple = [
                    random.randint(2, size_of_board - 2),
                    random.randint(2, size_of_board - 2),
                ]

            self.grow = True
        if self.head in self.wall or self.head in self.body[1:]:
            return False

    def draw(self, screen):
        # Draw the board
        for row in range(size_of_board):
            for col in range(size_of_board):
                # Calculate the position of the square and its border
                x = col * size_of_square
                y = row * size_of_square + height
                square_rect = pygame.Rect(x, y, size_of_square, size_of_square)
                border_rect = square_rect.inflate(-3, -3)

                # Draw the border and fill the square
                pygame.draw.rect(screen, (0, 0, 0), square_rect, 2)
                pygame.draw.rect(screen, (60, 60, 60), border_rect, 1)

        # Draw the snake's body
        for i, part in enumerate(self.body):
            rect = pygame.Rect(
                part[0] * size_of_square,
                part[1] * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            if i == 0:  # draw circle on head
                pygame.draw.circle(
                    screen, (0, 0, 255), rect.center, size_of_square // 2
                )
            elif i == len(self.body) - 1:  # draw triangle on last square
                x, y = rect.center
                triangle_points = [
                    (x, y - size_of_square // 2),
                    (x - size_of_square // 2, y + size_of_square // 2),
                    (x + size_of_square // 2, y + size_of_square // 2),
                ]
                pygame.draw.polygon(screen, (0, 0, 255), triangle_points)
            else:  # draw normal square
                pygame.draw.rect(screen, (0, 255, 0), rect)
            pygame.draw.rect(screen, (0, 0, 255), rect, 3)
        # Draw the walls
        for wall in self.wall:
            rect = pygame.Rect(
                wall[0] * size_of_square,
                wall[1] * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            pygame.draw.rect(screen, (0, 0, 0), rect)

        # Draw the apple
        rect = pygame.Rect(
            self.apple[0] * size_of_square,
            self.apple[1] * size_of_square + height,
            size_of_square,
            size_of_square,
        )
        pygame.draw.rect(screen, (255, 0, 0), rect)

    def gameOver(self, screen, new_screen_size):
        game_over = True
        # Capture a screenshot of the previous screen
        prev_screen = pygame.display.get_surface().copy()
        # Draw the game over screen
        screen.fill((255, 255, 255))
        # Blit the previous screen onto the background
        screen.blit(prev_screen, (0, 0))
        font = pygame.font.SysFont(None, 48)
        text = font.render("Game Over", True, (255, 0, 0))
        text_rect = text.get_rect(center=(screen_size[0] // 2, 40))
        screen.blit(text, text_rect)

        # Draw the restart button
        restart_button = pygame.Rect(
            new_screen_size[0] // 2 - 50, new_screen_size[1] // 2 + 50, 100, 50
        )
        pygame.draw.rect(screen, (0, 0, 255), restart_button)
        font = pygame.font.SysFont(None, 24)
        restart_text = font.render("Restart", True, (255, 255, 255))
        restart_text_rect = restart_text.get_rect(center=restart_button.center)
        screen.blit(restart_text, restart_text_rect)

        pygame.display.update()
        # print(self.wall)

        # Wait for user to click restart button or quit the game
        while game_over:
            for event in pygame.event.get():
                if (
                    event.type == pygame.MOUSEBUTTONDOWN
                    and restart_button.collidepoint(event.pos)
                ):
                    game_over = False
        # If the user clicked the restart button, restart the game

        main()

    def get_wall(self):
        return self.wall

    def draw_path(self, screen):
        path1 = []
        if not self.astar:
            return

        path = (
            self.astar.path
        )  # Change this line to use self.astar.path instead of self.astar.get_draw_path()
        for node in path:
            path1.append((node.x, node.y))

        if not path:
            return

        for node in path:
            rect = pygame.Rect(
                node.x * size_of_square,
                node.y * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            pygame.draw.rect(screen, (255, 255, 170), rect)
            pygame.draw.rect(screen, (0, 0, 255), rect, 3)



def execute(clock_tick, size_of_board, size_of_square, height):
    pygame.init()
    screen_size = (size_of_board * size_of_square, size_of_board * size_of_square + height)
    screen = pygame.display.set_mode(screen_size)
    pygame.display.set_caption("Snake A*")
    clock = pygame.time.Clock()

    snake = Snake()
    game_over = False  # Add a variable to track game over
    while True:
        grid = [
            [Node(x, y) for y in range(size_of_board)] for x in range(size_of_board)
        ]

        # Update the snake and draw it
        snake.update(grid)
        screen.fill((0, 0, 0))
        snake.draw_path(screen)  # Add this line to draw the path
        snake.draw(screen)
        pygame.display.flip()

        # Check if the snake has collided and end the game
        if snake.update(grid) == False:
            game_over = True  # Set game_over to True
            snake.gameOver(screen, screen_size)

        clock.tick(clock_tick)


if __name__ == "__main__":
    clock_tick = 100
    size_of_board = 60
    size_of_square = 12
    height = 20
    execute(clock_tick, size_of_board, size_of_square, height)



pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
!pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 7.0 MB/s eta 0:00:0000:0100:01


In [ ]:
import pygame
import random
import math


# Determine the constant sizes
size_of_board = 50
size_of_square = 12
height = 28
screen_size = (size_of_board * size_of_square, size_of_board * size_of_square + height)


# Creates a Node class
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.distance_from_start = 0
        self.heuristic = 0
        self.total_cost = 0

    # Determines whether or not the two coordinate values are equal
    def equal(self, compare):
        return self.x == compare.x and self.y == compare.y


class Algorithm:
    def __init__(self, grid):
        self.grid = grid
        self.queue = []
        self.explored_set = []
        self.path = []

    def get_initstate_and_fruit(self, snake):
        return Node(snake.head[0], snake.head[1]), Node(snake.apple[0], snake.apple[1])

    def manhattan_distance(self, nodeA, nodeB):
        distance_1 = abs(nodeA.x - nodeB.x)
        distance_2 = abs(nodeA.y - nodeB.y)
        return distance_1 + distance_2

    def get_path(self, node):
        if node.parent == None:
            return node

        while node.parent.parent != None:
            self.path.append(node)
            node = node.parent
        return node

    def inside_body(self, snake, node):
        for body in snake.body:
            if body[0] == node.x and body[1] == node.y:
                return True
        return False

    def outside_boundary(self, node):
        if 1 < node.x < size_of_board - 2:
            return False
        if 1 < node.y < size_of_board - 2:
            return False
        return True

    def get_neighbors(self, head):
        i = int(head.x)
        j = int(head.y)

        neighbors = []
        # left [i-1, j]
        if i > 1:
            neighbors.append(self.grid[i - 1][j])

        # right [i+1, j]
        if i < size_of_board - 2:
            neighbors.append(self.grid[i + 1][j])

        # top [i, j-1]
        if j > 1:
            neighbors.append(self.grid[i][j - 1])

        # bottom [i, j+1]
        if j < size_of_board - 2:
            neighbors.append(self.grid[i][j + 1])

        return neighbors


class A_STAR(Algorithm):
    def __init__(self, grid):
        super().__init__(grid)

    def run_algorithm(self, snake):
        # clear everything
        self.queue = []
        self.explored_set = []
        # to remove

        self.path = []

        snake_head, fruit = self.get_initstate_and_fruit(snake)

        self.new_path = None

        # open list
        self.queue.append(snake_head)

        # while we have states in open list
        while len(self.queue) > 0:
            # get node with lowest f(n)
            lowest_index = 0
            for i in range(len(self.queue)):
                if self.queue[i].total_cost < self.queue[lowest_index].total_cost:
                    lowest_index = i

            lowest_node = self.queue.pop(lowest_index)

            # check if it's the goal state
            if lowest_node.equal(fruit):
                self.new_path = self.get_path(lowest_node)
                return self.new_path

            self.explored_set.append(lowest_node)  # mark visited

            neighbors = self.get_neighbors(lowest_node)  # get neighbors

            # for each neighbor
            for neighbor in neighbors:

                # check if path is inside snake, outside boundary, or already visited
                if (
                    self.inside_body(snake, neighbor)
                    or self.outside_boundary(neighbor)
                    or neighbor in self.explored_set
                ):
                    continue  # skip this path
                # Updates the distance cost from the start
                temp = lowest_node.distance_from_start + 1

                best = False  # assuming neighbor path is better

                if neighbor not in self.queue:  # first time visiting
                    neighbor.heuristic = self.manhattan_distance(fruit, neighbor)
                    self.queue.append(neighbor)
                    best = True
                elif (
                    lowest_node.distance_from_start < neighbor.distance_from_start
                ):  # has already been visited
                    best = (
                        True  # but had a worse distance from the start, now it's better
                    )

                if best:
                    neighbor.parent = lowest_node
                    neighbor.distance_from_start = temp
                    neighbor.total_cost = (
                        neighbor.distance_from_start + neighbor.heuristic
                    )

        return None


class Snake:
    def __init__(self):
        self.body = [
            [size_of_board - 2, 1],
            [size_of_board - 3, 2],
            [size_of_board - 4, 3],
        ]
        self.head = self.body[0]
        self.apple = [
            random.randint(2, size_of_board - 2),
            random.randint(2, size_of_board - 2),
        ]
        self.direction = (1, 0)
        self.grow = False
        self.wall = []
        self.astar = None
        for i in range(size_of_board):
            self.wall.insert(0, [i, 0])
            self.wall.insert(0, [0, i])
            self.wall.insert(0, [i, size_of_board - 1])
            self.wall.insert(0, [size_of_board - 1, i])

    def update(self, grid):
        astar = A_STAR(grid)
        self.astar = astar
        next_move = astar.run_algorithm(self)
        if next_move:
            new_direction = (next_move.x - self.head[0], next_move.y - self.head[1])

            # Update the direction only if it is not the opposite of the current direction
            if (new_direction[0], new_direction[1]) != (
                -self.direction[0],
                -self.direction[1],
            ):
                self.direction = new_direction

        new_head = [self.head[0] + self.direction[0], self.head[1] + self.direction[1]]
        self.body.insert(0, new_head)
        self.head = new_head

        if not self.grow:
            self.body.pop()
        else:
            # print('body = \t', self.body, '\napple = ', self.apple, '\nself.head = ', self.head)

            self.grow = False

        if self.head == self.apple:
            self.apple = [
                random.randint(2, size_of_board - 2),
                random.randint(2, size_of_board - 2),
            ]
            while self.apple in self.body or self.apple in self.wall:
                self.apple = [
                    random.randint(2, size_of_board - 2),
                    random.randint(2, size_of_board - 2),
                ]

            self.grow = True
        if self.head in self.wall or self.head in self.body[1:]:
            return False

    def draw(self, screen):
        # Draw the board
        for row in range(size_of_board):
            for col in range(size_of_board):
                # Calculate the position of the square and its border
                x = col * size_of_square
                y = row * size_of_square + height
                square_rect = pygame.Rect(x, y, size_of_square, size_of_square)
                border_rect = square_rect.inflate(-3, -3)

                # Draw the border and fill the square
                pygame.draw.rect(screen, (0, 0, 0), square_rect, 2)
                pygame.draw.rect(screen, (100, 100, 100), border_rect, 1)

        # Draw the snake's body
        for i, part in enumerate(self.body):
            rect = pygame.Rect(
                part[0] * size_of_square,
                part[1] * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            if i == 0:  # draw circle on head
                pygame.draw.circle(
                    screen, (0, 0, 255), rect.center, size_of_square // 2
                )
            elif i == len(self.body) - 1:  # draw triangle on last square
                x, y = rect.center
                triangle_points = [
                    (x, y - size_of_square // 2),
                    (x - size_of_square // 2, y + size_of_square // 2),
                    (x + size_of_square // 2, y + size_of_square // 2),
                ]
                pygame.draw.polygon(screen, (0, 0, 255), triangle_points)
            else:  # draw normal square
                pygame.draw.rect(screen, (0, 255, 0), rect)
            pygame.draw.rect(screen, (0, 0, 255), rect, 3)
        # Draw the walls
        for wall in self.wall:
            rect = pygame.Rect(
                wall[0] * size_of_square,
                wall[1] * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            pygame.draw.rect(screen, (0, 0, 0), rect)

        # Draw the apple
        rect = pygame.Rect(
            self.apple[0] * size_of_square,
            self.apple[1] * size_of_square + height,
            size_of_square,
            size_of_square,
        )
        pygame.draw.rect(screen, (255, 0, 0), rect)

    def gameOver(self, screen):
        game_over = True
        # Capture a screenshot of the previous screen
        prev_screen = pygame.display.get_surface().copy()
        # Draw the game over screen
        screen.fill((255, 255, 255))
        # Blit the previous screen onto the background
        screen.blit(prev_screen, (0, 0))
        font = pygame.font.SysFont(None, 48)
        text = font.render("Game Over", True, (255, 0, 0))
        text_rect = text.get_rect(center=(screen_size[0] // 2, 40))
        screen.blit(text, text_rect)

        # Draw the restart button
        restart_button = pygame.Rect(
            screen_size[0] // 2 - 50, screen_size[1] // 2 + 50, 100, 50
        )
        pygame.draw.rect(screen, (0, 0, 255), restart_button)
        font = pygame.font.SysFont(None, 24)
        restart_text = font.render("Restart", True, (255, 255, 255))
        restart_text_rect = restart_text.get_rect(center=restart_button.center)
        screen.blit(restart_text, restart_text_rect)

        pygame.display.update()
        # print(self.wall)

        # Wait for user to click restart button or quit the game
        while game_over:
            for event in pygame.event.get():
                if (
                    event.type == pygame.MOUSEBUTTONDOWN
                    and restart_button.collidepoint(event.pos)
                ):
                    game_over = False
        # If the user clicked the restart button, restart the game

        main()

    def get_wall(self):
        return self.wall

    def draw_path(self, screen):
        path1 = []
        if not self.astar:
            return

        path = (
            self.astar.path
        )  # Change this line to use self.astar.path instead of self.astar.get_draw_path()
        for node in path:
            path1.append((node.x, node.y))

        if not path:
            return

        for node in path:
            rect = pygame.Rect(
                node.x * size_of_square,
                node.y * size_of_square + height,
                size_of_square,
                size_of_square,
            )
            pygame.draw.rect(screen, (128, 128, 128), rect)
            pygame.draw.rect(screen, (0, 0, 255), rect, 3)


def main():
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    pygame.display.set_caption("Snake A*")
    clock = pygame.time.Clock()

    snake = Snake()
    game_over = False  # Add a variable to track game over
    while True:
        grid = [
            [Node(x, y) for y in range(size_of_board)] for x in range(size_of_board)
        ]

        # Update the snake and draw it
        snake.update(grid)
        screen.fill((0, 0, 0))
        snake.draw_path(screen)  # Add this line to draw the path
        snake.draw(screen)
        pygame.display.flip()

        # Check if the snake has collided and end the game
        if snake.update(grid) == False:
            game_over = True  # Set game_over to True
            snake.gameOver(screen)

        clock.tick(100)


if __name__ == "__main__":
    main()


pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
